# Import libraries

In [1]:
# import libraries
import numpy as np
import sys
import os

from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

# for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
hfont = {'family':'sans-serif','fontname':'Helvetica', 'size':20}
mpl.rc('font',family='sans-serif', size=16)

# Define Functions

In [2]:
def fc1(rij, rc):
    f = (rij<rc)*np.tanh(1-rij/rc)**3
    return f

def fc2(rij, rc):
    f = (rij<rc)*0.5*(np.cos(np.pi*rij/rc)+1.0)
    return f

def G2(rij, rc, rs, eta, fc=fc1):
    f = fc(rij, rc)*np.exp(-eta*(rij-rs)**2)
    return f

def G3(rij, rik, rjk, cosine, rc, l, eta, epsilon, fc=fc1):
    f = 2.0**(1.0-epsilon)*fc(rij, rc)*fc(rik, rc)*fc(rjk, rc)*np.exp(-eta*(rij**2+rik**2+rjk**2))*(1.0+l*cosine)**epsilon
    return f

def G4(rij, rik, cosine, rc, l, eta, epsilon, fc=fc1):
    f = 2.0**(1.0-epsilon)*fc(rij, rc)*fc(rik, rc)*np.exp(-eta*(rij**2+rik**2))*(1.0+l*cosine)**epsilon
    return f

class Snapshot:
    def __init__(self, particles, box):
        self.particles = particles
        self.box = box
    def get_npart(self):
        return len(self.particles)
    def get_volume(self):
        return self.box[0]*self.box[1]*self.box[2]
    def get_density(self):
        return float(self.get_npart())/self.get_volume()


def read_snapshot(file_in):
    # read number of particles
    Npart = int(file_in.readline())
    # read box
    box = [float(n) for n in file_in.readline().split()]
    # skip line
    temp = file_in.readline()
    # read coordinates and types
    particles = []
    for i in range(Npart):
        temp = [n for n in file_in.readline().split()]
        xyz = [float(temp[0]), float(temp[1]), float(temp[2])]
        particles.append(xyz)
    # convert to np arrays
    box = np.array(box)
    particles = np.array(particles)
    # create snapshot
    snap = Snapshot(particles, box)
    return snap


def build_snapG3sfs(snap, rc, l_list, eta_list, eps_list):
    par = snap.particles
    box = snap.box
    Nsf = len(l_list)
    
    sf_par = np.zeros((len(par), Nsf))
    for i, p in enumerate(par):
        dist = par - p
        dist -= box*np.rint(dist/box)
        dr = np.linalg.norm(dist, axis=1)
        neigh_dist = np.array([dj for dj, rj in zip(dist,dr) if (rj<rc and rj>0)])
        neigh_dr = np.array([rj for rj in dr if (rj<rc and rj>0)])
        Nneigh = len(neigh_dr)
        temp_trip_dr = [[neigh_dr[j], neigh_dr[k]] for j in range(Nneigh) for k in range(j+1,Nneigh)]
        temp_trip_dist = np.array([[neigh_dist[j], neigh_dist[k]] for j in range(Nneigh) for k in range(j+1,Nneigh)])
        temp_rjk = []
        if len(temp_trip_dist>0):
            temp_rjk = np.linalg.norm(temp_trip_dist[:,1]-temp_trip_dist[:,0], axis=1)
        
        rij = np.array([t[0] for t,r in zip(temp_trip_dr, temp_rjk) if r<rc])
        dij = np.array([d[0] for d,r in zip(temp_trip_dist, temp_rjk) if r<rc])
        rik = np.array([t[1] for t,r in zip(temp_trip_dr, temp_rjk) if r<rc])
        dik = np.array([d[1] for d,r in zip(temp_trip_dist, temp_rjk) if r<rc])
        rjk = np.array([r for r in temp_rjk if r<rc])
        if len(rij)>0:
            cosine = np.sum(dij*dik, axis=1)/(rij*rik)
            count = 0
            for (l, eta, eps) in zip(l_list, eta_list, eps_list):
                sf_par[i][count] = np.sum(G3(rij, rik, rjk, cosine, rc, l, eta, eps))
                count += 1
    sf_snap = np.sum(sf_par, axis=0)
    return sf_snap

# Choose parameters

In [3]:
# size ratio
q = 0.4

# build sets of parameters for G3 symmetry functions in the pool of candidates
rc_set = np.array([1.+q])
l_set = np.array([1.0,-1.0])
eta_set = np.array([0.001, 0.01, 0.1, 1, 2, 4, 8])
eps_set = np.array([1, 2, 4, 8])

# build complete list of parameters
rc_list = [rc for rc in rc_set for l in l_set for eta in eta_set for eps in eps_set]
l_list = [l for rc in rc_set for l in l_set for eta in eta_set for eps in eps_set]
eta_list = [eta for rc in rc_set for l in l_set for eta in eta_set for eps in eps_set]
eps_list = [eps for rc in rc_set for l in l_set for eta in eta_set for eps in eps_set]
npool = len(rc_list)

print('Size ratio q:   ' + str(q))
print('Pool dimension: ' + str(npool))  

Size ratio q:   1.0
Pool dimension: 56


# Build pool of candidates

In [9]:
directory = 'states/trajectories'
file_names = os.listdir(directory)
file_names.sort()
nsnaps = 500

pool = np.zeros((nsnaps*len(file_names),npool))
count = 0
for n, file_name in enumerate(file_names):
    file_name = os.path.join(directory, file_name)
    print('(%3d/%3d) Processing file: %s' % (n+1, len(file_names), file_name))
    file_in = open(file_name, 'r')
    for i in range(nsnaps):
        snap = read_snapshot(file_in)
        pool[count] = build_snapG3sfs(snap, q+1, l_list, eta_list, eps_list)
        count += 1
    file_in.close()

print('\nPool of candidates shape: ' + str(pool.shape))

# save dataset
folder = './Dataset_q%.2f' % (q)
if not os.path.exists(folder):
    os.mkdir(folder)
save_path = folder + '/pool.npy'
np.save(save_path, pool)

(  1/ 99) Processing file: states/trajectories/TRJ_.305577491.xyz
(  2/ 99) Processing file: states/trajectories/TRJ_.315126788.xyz
(  3/ 99) Processing file: states/trajectories/TRJ_.324676085.xyz
(  4/ 99) Processing file: states/trajectories/TRJ_.334225382.xyz
(  5/ 99) Processing file: states/trajectories/TRJ_.343774679.xyz
(  6/ 99) Processing file: states/trajectories/TRJ_.353323976.xyz
(  7/ 99) Processing file: states/trajectories/TRJ_.362873273.xyz
(  8/ 99) Processing file: states/trajectories/TRJ_.372422570.xyz
(  9/ 99) Processing file: states/trajectories/TRJ_.381971867.xyz
( 10/ 99) Processing file: states/trajectories/TRJ_.391521164.xyz
( 11/ 99) Processing file: states/trajectories/TRJ_.401070461.xyz
( 12/ 99) Processing file: states/trajectories/TRJ_.410619758.xyz
( 13/ 99) Processing file: states/trajectories/TRJ_.420169055.xyz
( 14/ 99) Processing file: states/trajectories/TRJ_.429718352.xyz
( 15/ 99) Processing file: states/trajectories/TRJ_.439267649.xyz
( 16/ 99) 

# Build target

In [5]:
idx = int(np.rint(q/0.05)) - 4
directory = 'states/energies'
file_names = os.listdir(directory)
file_names.sort()
nsnaps = 500

E = []
for n, file_name in enumerate(file_names):
    file_name = os.path.join(directory, file_name)
    file_in = open(file_name, 'r')
    lines = file_in.readlines()
    lines = lines[1:]
    data = np.array([line.split() for line in lines], dtype=float)
    data = data[:,idx+1]
    E += list(data[:nsnaps])
E = np.array(E)
print('\nTraget shape: ' + str(E.shape))

# save dataset
folder = './Dataset_q%.2f' % (q)
if not os.path.exists(folder):
    os.mkdir(folder)
save_path = folder + '/target.npy'
np.save(save_path, E)


Traget shape: (49500,)
